<a href="https://colab.research.google.com/github/pin164/Automated-Expense-Categorization/blob/main/Decision_Tree_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STEP 1
# Import  dependencies
!pip install category_encoders
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn import tree
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
import category_encoders as ce

from google.colab import drive
drive.mount('/content/drive')



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 3.6 MB/s eta 0:00:00
Mounted at /content/drive


## Preprocessing

In [2]:
# STEP 2
#  Import and read the Cleaned_Invoice_4.csv
#/content/drive/MyDrive/Project 4/Automated-Expense-Categorization/Cleaned_Invoice_4.csv
application_df = pd.read_csv("/content/drive/MyDrive/Project 4/Automated-Expense-Categorization/Cleaned_Invoice_5.csv",encoding='latin1').iloc[:,:14]
application_df.head()

,Vendor,Month,Year,PO#,Invoice Date,Invoice#,total Invoice Amt,TRS,Project,Amount,Cost Centre,Order#,Dept
0,Vendor_1024,Jan,2022,R4501240857,2022-01-31,IS-232223,"$10,726.38",SDS7089359,TCF134 - Robotic Process Automation Sco,"$2,992.63",214347,R100012873,FIN
1,Vendor_1024,Feb,2022,R4501240857,2022-02-28,IS-232939,"$18,527.38",SDS7089359,TCF134 - Robotic Process Automation Sco,"$2,286.50",214347,R100012873,FIN
2,Vendor_1024,Jun,2022,R4501253248,2022-06-30,IS-236635,"$20,477.63",SDS7089359,TCF134 - Robotic Process Automation Sco,$269.00,214347,R100012873,FIN
3,Vendor_1024,Jul,2022,R4501253248,2022-07-31,IS-237624,"$19,502.50",SDS7089359,TCF134 - Robotic Process Automation Sco,"$4,707.50",214347,R100012873,FIN
4,Vendor_1024,Aug,2022,R4501253248,2022-08-31,IS-238521,"$21,452.75",SDS7089359,TCF134 - Robotic Process Automation Sco,"$6,792.25",214347,R100012873,FIN


In [3]:
# STEP 3
# extract beneficial columns only
####X_df = application_df[['Column2','PO#','TRS','Order#','Column3']]
X_df = application_df[['PO#','TRS','Order#']]
Y_df = application_df['Cost Centre']



In [4]:
#Step 4
# amend the data type of order to be an object
X_df.loc[:, 'Order#'] = X_df['Order#'].astype('object')

X_df.dtypes

,0
PO#,object
TRS,object
Order#,object


In [5]:
#Step 5
# Ordinal encoding transforms the columns PO#','TRS','Order#'
# into a column of class integers.
# Ordinal encoding scheme maps columns to integers randomly


encoder = ce.OrdinalEncoder()
encoder.fit_transform(X_df).head()

,PO#,TRS,Order#
0,1,1,1
1,1,1,1
2,2,1,1
3,2,1,1
4,2,1,1


In [6]:
X_encoded = encoder.fit_transform(X_df) # Store the encoded DataFrame in X_encoded

In [7]:
#Drop Rows with Missing Target Values:
#remove the rows from both the feature set X and the target set Y_df that contain missing target values.
# Combine X and y to drop rows with missing target values
combined_df = pd.concat([X_encoded, Y_df], axis=1)

# Drop rows where the target variable is missing
combined_df.dropna(subset=['Cost Centre'], inplace=True)

# Separate the features and target again after dropping missing values
X = combined_df.drop(columns=['Cost Centre']).values
y = combined_df['Cost Centre'].values


# Re-split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets to verify the split
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Target Shape:", y_test.shape)



Training Features Shape: (5629, 3)
Testing Features Shape: (1408, 3)
Training Target Shape: (5629,)
Testing Target Shape: (1408,)


In [8]:
#  Step 6: Standardize (scale) the feature data
scaler = StandardScaler()

# Fit on training data, transform both training and test sets
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)
X_test_scaled = scaler.transform(X_test).astype(np.float32)

In [9]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
rf_model = RandomForestClassifier(random_state=42)

In [10]:
#Some Cost Centres dominate the dataset,
# the model could struggle to predict minority payees.
# Using Class Weights:
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
# fit the model
rf_model.fit(X_train, y_train)
# Prediction
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=1))



              precision    recall  f1-score   support

      110508       1.00      1.00      1.00        10
      110889       1.00      1.00      1.00         9
      111116       1.00      1.00      1.00         1
      111173       1.00      1.00      1.00        13
      111508       1.00      1.00      1.00        14
      135072       1.00      1.00      1.00        14
      135139       1.00      1.00      1.00         6
      135150       1.00      1.00      1.00        12
      135367       1.00      1.00      1.00         9
      140010       1.00      1.00      1.00        20
      140548       1.00      1.00      1.00        29
      160207       1.00      1.00      1.00         4
      160635       1.00      1.00      1.00         3
      160781       1.00      1.00      1.00         1
      160996       1.00      1.00      1.00         4
      161017       1.00      1.00      1.00        14
      163124       1.00      1.00      1.00         4
      165031       1.00    

In [11]:
rf_confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(rf_confusion)

Confusion Matrix:
[[10  0  0 ...  0  0  0]
 [ 0  9  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  6  0  0]
 [ 0  0  0 ...  0  7  0]
 [ 0  0  0 ...  0  0  5]]


In [12]:
# clf is your trained RandomForestClassifier
import joblib
joblib.dump(rf_model, 'rf_model.pkl')

['rf_model.pkl']

In [13]:
# Save encoder as well the way we saved the model
joblib.dump(encoder, 'category_encoder.pkl')



['category_encoder.pkl']

streamlit code

In [14]:
## create a def

In [15]:
import pandas as pd
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
import joblib
# Load the model from the file
clf_loaded = joblib.load('/content/rf_model.pkl')
# Load the encoder from the file
encoder_loaded = joblib.load('/content/category_encoder.pkl')

In [16]:
clf_loaded

RandomForestClassifier(class_weight='balanced', random_state=42)

In [17]:
encoder_loaded

OrdinalEncoder(cols=['PO#', 'TRS', 'Order#'],
               mapping=[{'col': 'PO#', 'data_type': dtype('O'),
                         'mapping': R4501240857      1
R4501253248      2
R4501128230      3
R4501167712      4
R4501179019      5
              ... 
R4501290139    427
R4501233935    428
R4501235275    429
R4501258933    430
NaN             -2
Length: 431, dtype: int64},
                        {'col': 'TRS', 'data_type': dtype('O'),
                         'mapping': SDS7089359      1
SDS7089920      2
SDS7090297      3
SDS7089945      4
SDS7090154      5
             ... 
SDS7090401    197
SDS7090403    198
SDS7090402    199
SDS7090432    200
NaN            -2
Length: 201, dtype: int64},
                        {'col': 'Order#', 'data_type': dtype('O'),
                         'mapping': R100012873      1
R100013089      2
R100013400      3
R100013663      4
R100013680      5
             ... 
R930092877    167
R930092879    168
R930093000    169
R930093243    170
NaN            -2
Length: 171, dtype: int64}])

In [18]:
test =pd.DataFrame(columns= ['PO#','TRS','Order#'])

step one in streamlit

In [25]:
po = input("Enter PO#: ")
trs = input("Enter TRS: ")
order = input("Enter Order#: ")


Enter PO#: R4501207601
Enter TRS: SDS7089920
Enter Order#: R100013089


In [20]:
test.loc[0] = [po,trs,order]

In [21]:
test

,PO#,TRS,Order#
0,R4501186291,SDS7089920,R100013089


In [22]:
# Assuming you have new data (X_test) to transform
X_test_transformed = encoder_loaded.transform(test)

In [23]:
X_test_transformed

,PO#,TRS,Order#
0,6,2,2


In [24]:
clf_loaded.predict(X_test_transformed)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


array([171225])